In [152]:
import os
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

root = Path(os.getcwd()).parent.parent

#Change for each one of where your data is. For me in Dataset I have all the three folders
parent_directory = os.path.dirname(root)
dataset_path = os.path.join(root,'Dataset')

ADVOCATE = "advocate"
RATEBEER = "ratebeer"
MATCHED = "matched"
FULL = "full"

ADVOCATE_PATH = os.path.join(dataset_path,ADVOCATE)
RATEBEER_PATH = os.path.join(dataset_path,RATEBEER)
MATCHED_PATH = os.path.join(dataset_path,MATCHED)
FULL_PATH = os.path.join(dataset_path,FULL)

**Introduction**

The goal of this notebook is to found a way to transform every family dataset into one single dataset. At the end we should have one single user dataset, one single beer dataset and so on. Those datasets would be downloadable, except ratings where we want to create a function that we can run at the beginning of each script, this is done to avoid to have to download/load a much heavier dataset.

To facilitate our life we want to create a single id that could correspond accross the different datasets. It is important that there remains a way to connect to the dataset of origin, i.e that we do not drop the general id.

In [199]:
#Open this one already as it may become usefull for reweries already
beers_matched = pd.read_csv(os.path.join(MATCHED_PATH,'beers.csv'),header=1)

**Breweries dataset**

We saw that in data_understanding.ipynb, for breweries there were duplicates. So we need to treat them carefully.

In [200]:
breweries_advocate = pd.read_csv(os.path.join(ADVOCATE_PATH,'breweries.csv'))
breweries_matched = pd.read_csv(os.path.join(MATCHED_PATH,'breweries.csv'),header=1)
breweries_ratebeer = pd.read_csv(os.path.join(RATEBEER_PATH,'breweries.csv'))

print("Length of the three datasets:\n-advocate:",len(breweries_advocate),"\n-matched:", len(breweries_matched),"\n-ratebeer:", len(breweries_ratebeer))

#Already delete the breweries presents in matched
breweries_ratebeer_solo = breweries_ratebeer[~breweries_ratebeer.id.isin(breweries_matched['id.1'])]
breweries_advocate_solo = breweries_advocate[~breweries_advocate.id.isin(breweries_matched['id'])]

Length of the three datasets:
-advocate: 16758 
-matched: 8281 
-ratebeer: 24189


Both advocate and ratebeer dataset have as columns ['id', 'location', 'name', 'nbr_beers']. Matched has those columns with .1 referring to ratebeer, they also have a diff and sim column. The end format we want to have: [general_id, old_id_advocate, 'old_id_ratebeer', location, name and nbr_beers]. However we need to be carefull to the duplicates of beers in the beers dataset. As the duplicates where for advocate and not Ratebeer we make the convention to take the name and location from Ratebeer. We also want to drop diff and sim.  
Let us first get the duplicates of breweries.

In [201]:
breweries_duplicates_ratebeer = breweries_matched[breweries_matched['id.1'].duplicated(keep=False)]
breweries__not_duplicated_ratebeer = breweries_matched[~breweries_matched['id.1'].duplicated(keep=False)]#For later
breweries_duplicates_advocate = breweries_matched[breweries_matched['id'].duplicated(keep=False)]

print("Number of duplicated/tripled Ratbeer breweries",len(breweries_duplicates_ratebeer))
print("Number of duplicated/tripled Advocate breweries",len(breweries_duplicates_advocate))

Number of duplicated/tripled Ratbeer breweries 91
Number of duplicated/tripled Advocate breweries 0


In [202]:
#We are going to build a new dataset instead of playing with the dataset that contains every information for duplicates.

unique_ratebeer_brewery_id = breweries_duplicates_ratebeer['id.1'].unique()
new_matched_brewery_duplicate = pd.DataFrame(columns=['location', 'name', 'nbr_beers','old_advocate_id','old_ratebeer_id'])

for id in unique_ratebeer_brewery_id:  # Replace with your actual loop condition
    # Define data for each row
    location = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['location.1'].iloc[0]#Always take the first as it is the same for the two/three of them
    name = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['name.1'].iloc[0]#Always take the first as it is the same for the two/three of them
    old_advocate_id = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['id']
    list_old_advocate_id = []
    for id_advocate in (old_advocate_id):
        list_old_advocate_id.append(id_advocate)
    old_ratebeer_id = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['id.1'].iloc[0]

    total_beers_advocate = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['nbr_beers'].sum()
    total_beers_ratebeer = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['nbr_beers.1'].sum()

    nbr_beers = total_beers_advocate+ total_beers_ratebeer

    # Append the new row to the DataFrame
    new_row = pd.DataFrame({'location': location, 'name': name, 'nbr_beers': nbr_beers,'old_advocate_id':[list_old_advocate_id],'old_ratebeer_id':old_ratebeer_id})
    new_matched_brewery_duplicate = pd.concat([new_matched_brewery_duplicate, new_row], ignore_index=True)

new_matched_brewery_duplicate.sample(10)


,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
11,Russia,Tinkoff,39,"[621, 2357]",3874
30,"United States, Kansas",Wichita Brewing Company & Pizzeria,188,"[40365, 26634]",13208
40,"United States, Illinois",Obed and Isaacs Microbrewery and Eatery,130,"[28569, 48215]",14267
17,"United States, Louisiana",Rikenjaks Brewing Company,10,"[44929, 970]",3860
43,France,Goutte dOr,59,"[34186, 34365]",16083
28,"United States, Kentucky",Red Ear Brewing Company,12,"[25151, 24451]",11761
10,Germany,Brauhaus am Kreuzberg &#40;Friedels Keller&#41;,111,"[5809, 20507]",1498
26,"United States, Virginia",Crooked Run Brewing,198,"[32661, 48463]",17177
12,Australia,Cavalier Beer,174,"[25867, 36326]",12657
15,Sweden,Carlsberg Sverige,421,"[5368, 10897]",765


Did the next cell in a dumb way just need to rename the columns, would be faster.

In [203]:
unique_ratebeer_brewery_id = breweries__not_duplicated_ratebeer['id.1'].unique()
new_matched_brewery_non_duplicate = pd.DataFrame(columns=['location', 'name', 'nbr_beers','old_advocate_id','old_ratebeer_id'])

for id in unique_ratebeer_brewery_id:  # Replace with your actual loop condition
    # Define data for each row
    location = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['location.1'].iloc[0]#Always take the first as it is the same for the two/three of them
    name = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['name.1'].iloc[0]#Always take the first as it is the same for the two/three of them
    old_advocate_id = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['id'].iloc[0]
    old_ratebeer_id = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['id.1'].iloc[0]

    total_beers_advocate = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['nbr_beers'].sum()
    total_beers_ratebeer = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['nbr_beers.1'].sum()

    nbr_beers = total_beers_advocate+ total_beers_ratebeer

    # Append the new row to the DataFrame
    new_row = pd.DataFrame({'location': location, 'name': name, 'nbr_beers': nbr_beers,'old_advocate_id':[old_advocate_id],'old_ratebeer_id':old_ratebeer_id})
    new_matched_brewery_non_duplicate = pd.concat([new_matched_brewery_non_duplicate, new_row], ignore_index=True)

new_matched_brewery_non_duplicate.sample(3)

,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
6277,"United States, Florida",Fort Myers Brewing Company,70,31108,16272
1958,Germany,Brauerei Bruckmüller,19,11285,5271
1236,Japan,Satsuma Beer,13,9117,5919


In [204]:
#Concat the two datasets together
new_matched_brewery = pd.concat([new_matched_brewery_non_duplicate, new_matched_brewery_duplicate], ignore_index=True)
new_matched_brewery.sample(3)

,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
5781,"United States, Minnesota",Enki Brewing Company,49,31854,16776
693,Mexico,Ensenada Brewing Co.,10,33374,20309
4156,"United States, California",Sundowner Brewery,48,35011,16374


In [205]:
print(len(new_matched_brewery), len(new_matched_brewery_non_duplicate), len(new_matched_brewery_duplicate))
print(len(new_matched_brewery.old_ratebeer_id.unique()))

8235 8190 45
8235


Now we need to count the number of beers that there really is. We assume that a beer can only be matched if its brewery is matched too.

In [ ]:
new_matched_brewery_good_amount_rating = new_matched_brewery.copy()
for id in beers_matched['brewery_id.1']:
    new_matched_brewery_good_amount_rating.loc[new_matched_brewery_good_amount_rating['old_ratebeer_id'] == id, 'nbr_beers'] -= 1
    


In [207]:
new_matched_brewery_good_amount_rating.sample(3)

,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
2772,Spain,Cerveses Almogàver,7,29519,11501
446,England,Deeply Vale,20,37168,16023
7945,"United States, Vermont",Liftline Brewing Company,5,40534,23141


Now let's match the three datasets together.

In [208]:
breweries_ratebeer_solo = breweries_ratebeer_solo.rename(columns={'id': 'old_ratebeer_id'})
breweries_ratebeer_solo['old_advocate_id'] = np.nan
breweries_ratebeer_solo.sample(3)  

,old_ratebeer_id,location,name,nbr_beers,old_advocate_id
15513,11251,"United States, Colorado",Crazy Mountain Brewing Company,137,NaN
20666,12646,England,Hopping Mad,11,NaN
24117,31956,Thailand,Hercules,1,NaN


In [209]:
breweries_advocate_solo = breweries_advocate_solo.rename(columns={'id': 'old_advocate_id'})
breweries_advocate_solo['old_ratebeer_id'] = np.nan
breweries_advocate_solo.sample(3)  

,old_advocate_id,location,name,nbr_beers,old_ratebeer_id
3212,44189,Italy,Brunz Birreria Artigianale,2,NaN
10442,46489,"United States, Maryland",Chester River Brewing Company,7,NaN
4764,6134,Germany,Privatbrauerei Ludwig Würth,1,NaN


In [210]:
full_breweries = pd.concat([new_matched_brewery_good_amount_rating,breweries_advocate_solo, breweries_ratebeer_solo], ignore_index=True)
full_breweries['id'] = range(1, len(full_breweries) + 1)
full_breweries.sample(10)


,location,name,nbr_beers,old_advocate_id,old_ratebeer_id,id
18504,Bulgaria,Burgaska Pivovarna - Pivovarnata,5,NaN,24081,18505
20664,Germany,Bierserker &#40;Siegburger Abteibrauerei&#41;,9,NaN,27353,20665
13724,"United States, Pennsylvania",Roy Pitz Barrel House,0,49398,NaN,13725
22095,Russia,Hophead Craft Brewery,34,NaN,24671,22096
24446,South Africa,Bosheuwel Country Estate,4,NaN,21712,24447
29001,"United States, Ohio",Trailhead Brewery,9,NaN,18103,29002
17664,Canada,Yellowbelly Brewery and Public House,22,NaN,9635,17665
28094,"United States, Indiana",Wood Boat Brewing Company,6,NaN,17220,28095
10174,Germany,Gut Basthorst,1,34615,NaN,10175
16772,China,China Resources Snow Breweries (CRB) (SABMiller),39,NaN,5227,16773


In [211]:
print(len(full_breweries))
print(len(breweries_matched),len(breweries_advocate),len(breweries_ratebeer))
print(-len(breweries_matched)+len(breweries_advocate)+len(breweries_ratebeer))

32666
8281 16758 24189
32666


This confirms the number of single breweries we computed in data_understanding.ipynb.

In [212]:
full_breweries.to_csv(os.path.join(FULL_PATH,'breweries.csv'),index=False)

In [157]:
test_breweri = pd.read_csv(os.path.join(FULL_PATH,'breweries.csv'))
print(len(test_breweri))
del test_breweri

32666


**Users dataset**

We prefer to consider only the users not users approx. We saw that ths users do not have any duplicates (unlike users approx).

In [213]:
users_matched = pd.read_csv(os.path.join(MATCHED_PATH,'users.csv'),header=1)
users_advocate = pd.read_csv(os.path.join(ADVOCATE_PATH,'users.csv'))
users_ratebeer = pd.read_csv(os.path.join(RATEBEER_PATH,'users.csv'))

ratings_matched = pd.read_csv(os.path.join(MATCHED_PATH,'ratings.csv'),header=1)#needed 


print("Length of the three datasets:\n-advocate:",len(users_advocate),"\n-matched:", len(users_matched),"\n-ratebeer:", len(users_ratebeer))

#Already delete the breweries presents in matched
users_ratebeer_solo = users_ratebeer[~users_ratebeer.user_id.isin(users_matched['user_id.1'])]
users_advocate_solo = users_advocate[~users_advocate.user_id.isin(users_matched['user_id'])]

Length of the three datasets:
-advocate: 153704 
-matched: 3020 
-ratebeer: 70174


Both advocate and ratebeer dataset have as columns ['user_id', 'location', 'user_name','user_name_lower', 'joined','nbr_ratings']. Advocate has nbr_reviews in addition. Matched has those columns with .1 referring to ratebeer. The end format we want to have: [general_id, old_user_id_advocate, 'old_user__id_ratebeer', location, user_name_lower, joined_advocate, joined_ratebeer and nbr_ratings]. We think that the other columns will not be usefull for our analysis. The next cell shows that the user_name_lower are equal between the two datasets. For the location we will once again give prioruty to the ratebeer dataset.

In [214]:
print(len(users_matched[users_matched['user_name_lower']==users_matched['user_name_lower.1']]))
print(len(users_matched))

3020
3020


In [215]:
users_matched.sample(2)

,joined,location,nbr_ratings,nbr_reviews,user_id,user_name,user_name_lower,joined.1,location.1,nbr_ratings.1,user_id.1,user_name.1,user_name_lower.1
219,1.152785e+09,"United States, Florida",2,2,homer321.88471,homer321,homer321,1.200740e+09,"United States, Florida",4227,67955,Homer321,homer321
1601,1.338458e+09,"United States, Wisconsin",11,11,bshotbox.678984,bshotbox,bshotbox,1.296558e+09,"United States, Wisconsin",10,122694,bshotbox,bshotbox


In [240]:
users_matched_not_duplicated = users_matched.drop(columns=['nbr_reviews','user_name','user_name_lower','location','user_name_lower.1'])
users_matched_not_duplicated = users_matched_not_duplicated.rename(columns={'joined': 'joined_advocate','joined.1': 'joined_ratebeer','user_id': 'old_user_id_advocate','user_id.1': 'old_user_id_ratebeer','user_name.1':'user_name','location.1':'location'})
users_matched_not_duplicated["nbr_ratings_total"] = users_matched_not_duplicated.nbr_ratings+users_matched_not_duplicated["nbr_ratings.1"]
users_matched_not_duplicated = users_matched_not_duplicated.drop(columns=['nbr_ratings','nbr_ratings.1'])
users_matched_not_duplicated.sample(5)

,joined_advocate,old_user_id_advocate,joined_ratebeer,location,old_user_id_ratebeer,user_name,nbr_ratings_total
1294,1.322132e+09,kaga.639092,1.218881e+09,"United States, Indiana",80347,Kaga,5
1020,1.215425e+09,ttrrooyy.233039,1.256378e+09,"United States, South Carolina",96821,ttrrooyy,2
1661,1.390388e+09,mcwop.777661,1.379844e+09,"United States, Maryland",280375,McWop,25
1120,1.400148e+09,marcomignola.800071,1.400234e+09,Italy,318081,MarcoMignola,9
1163,1.494670e+09,slufsar.1200099,1.494670e+09,Netherlands,465525,slufsar,21


In [241]:
users_matched_not_duplicated_good_amount_ratings = users_matched_not_duplicated.copy()
for id in ratings_matched['user_id.1']:
    users_matched_not_duplicated_good_amount_ratings.loc[users_matched_not_duplicated_good_amount_ratings['old_user_id_ratebeer'] == id, 'nbr_ratings_total'] -= 1

print(users_matched_not_duplicated.nbr_ratings_total.sum())
print(users_matched_not_duplicated_good_amount_ratings.nbr_ratings_total.sum())

1526910
1504946


In [242]:
users_ratebeer_solo = users_ratebeer_solo.rename(columns={'user_id': 'old_user_id_ratebeer','joined':'joined_ratebeer','nbr_ratings':'nbr_ratings_total'})
users_ratebeer_solo['old_user_id_advocate'] = np.nan
users_ratebeer_solo['joined_advocate'] = np.nan
users_ratebeer_solo.sample(3)  

,nbr_ratings_total,old_user_id_ratebeer,user_name,joined_ratebeer,location,old_user_id_advocate,joined_advocate
53276,1,321483,Iannowak,1.402308e+09,NaN,NaN,NaN
66331,4,426144,Delniels,1.472551e+09,Portugal,NaN,NaN
39779,4,73601,LeiniesLover,1.208426e+09,"United States, California",NaN,NaN


In [246]:
users_advocate_solo = users_advocate_solo.rename(columns={'user_id': 'old_user_id_advocate','joined':'joined_advocate','nbr_ratings':'nbr_ratings_total'})
users_advocate_solo = users_advocate_solo.drop(columns=['nbr_reviews'])
users_advocate_solo['old_user_id_ratebeer'] = np.nan
users_advocate_solo['joined_ratebeer'] = np.nan
users_advocate_solo.sample(3)  

,nbr_ratings_total,old_user_id_advocate,user_name,joined_advocate,location,old_user_id_ratebeer,joined_ratebeer
52471,8,czoch75.75059,czoch75,1.145354e+09,"United States, Illinois",NaN,NaN
54659,6,msloosedag.902597,msloosedag,1.417604e+09,"United States, Pennsylvania",NaN,NaN
80347,13,s2000.798206,S2000,1.399198e+09,"United States, New Jersey",NaN,NaN


In [ ]:
full_users = pd.concat([users_matched_not_duplicated_good_amount_ratings,users_ratebeer_solo, users_advocate_solo], ignore_index=True)
full_users['id'] = range(1, len(full_users) + 1)
full_users.sample(2)


,joined_advocate,old_user_id_advocate,joined_ratebeer,location,old_user_id_ratebeer,user_name,nbr_ratings_total,id
152073,1.229252e+09,wessells.276935,NaN,"United States, California",NaN,wessells,1,152074
71636,1.373969e+09,ari32.742061,NaN,"United States, Georgia",NaN,ari32,876,71637


In [248]:
print(len(full_users))
print(len(users_matched),len(users_advocate),len(users_ratebeer))
print(-len(users_matched)+len(users_advocate)+len(users_ratebeer))

220858
3020 153704 70174
220858


In [249]:
full_users.to_csv(os.path.join(FULL_PATH,'users.csv'),index=False)

In [250]:
test_user = pd.read_csv(os.path.join(FULL_PATH,'users.csv'))
print(len(test_user))
del test_user

220858


**Beer dataset**

In this dataset we saw that there were no duplicates. We do not need to make the carefull analysis we made before. However we still need to look at the matched dataset. We also need to link the new brewery id.

In [6]:
beers_matched = pd.read_csv(os.path.join(MATCHED_PATH,'beers.csv'),header=1)
beers_advocate = pd.read_csv(os.path.join(ADVOCATE_PATH,'beers.csv'))
beers_ratebeer = pd.read_csv(os.path.join(RATEBEER_PATH,'beers.csv'))

print(len(beers_matched))
print(len(beers_advocate))
print(len(beers_ratebeer))

45640
280823
442081


In [ ]:
beers_matched.sample(10)

,abv,avg,avg_computed,avg_matched_valid_ratings,ba_score,beer_id,beer_name,beer_wout_brewery_name,brewery_id,brewery_name,...,brewery_id.1,brewery_name.1,nbr_matched_valid_ratings.1,nbr_ratings.1,overall_score,style.1,style_score,zscore.1,diff,sim
16147,5.7,4.07,3.883333,4.150000,NaN,97173,Extraterrestrial Space Beer,Extraterrestrial Space Beer,32092,Intergalactic Brewing Co.,...,17642,Intergalactic Brewing Company,1,1,NaN,Premium Bitter/ESB,NaN,-0.195097,0.802319,1.000000
15834,12.0,4.23,4.233333,4.950000,NaN,145222,AleSmith Speedway Stout - Honey Graham,Graham Speedway Honey Stout,396,AleSmith Brewing Company,...,432,AleSmith Brewing Company,5,5,NaN,Imperial Stout,NaN,0.691466,0.722944,1.000000
7954,5.6,3.56,3.540526,3.480000,83.0,98054,Ratsherrn Pale Ale,Pale Ale,31793,Ratsherrn Brauerei GmbH,...,14561,Ratsherrn Brauerei,182,182,45.0,American Pale Ale,37.0,-0.541521,0.731194,1.000000
42941,6.0,3.61,3.585614,3.475000,83.0,129335,Arjuna,Arjuna,30164,Anthem Brewing Company,...,14617,Anthem Brewing Company,29,29,69.0,Witbier,92.0,-0.091560,0.460728,1.000000
40569,6.2,3.64,3.531316,3.680000,83.0,83345,Beardy Guard,Guard Beardy,29438,Rogness Brewing Company,...,14294,Rogness Brewing Company,29,29,47.0,Bière de Garde,50.0,-0.461041,1.000000,1.000000
9202,4.6,3.24,3.240000,3.240000,NaN,48754,Effen Lager,Lager Effen,14014,Southern Bay Brewing Company,...,8056,Southern Bay Brewing Company,10,10,29.0,Pale Lager,98.0,-0.931210,0.448773,0.808393
3362,5.4,NaN,NaN,NaN,NaN,155915,Équinoxe Pale Ale,Pale Ale Équinoxe,1141,Brasserie Dieu du Ciel!,...,364,Dieu du Ciel,2,2,NaN,American Pale Ale,NaN,-0.044463,0.426658,1.000000
11123,0.5,1.65,1.650000,1.650000,NaN,89257,Royal Club Shandy,Royal Shandy Club,81,Heineken Nederland B.V.,...,9,Heineken Nederland,18,18,5.0,Low Alcohol,59.0,-2.374874,0.438918,1.000000
2529,4.1,3.41,3.472000,3.380000,NaN,63691,THAT,THAT,9568,Teme Valley Brewery,...,3086,Teme Valley,58,58,27.0,Bitter,30.0,-0.832641,1.000000,1.000000
20455,7.2,4.51,4.409186,4.532941,94.0,255380,Party Wave,Party Wave,26676,Kane Brewing Company,...,13267,Kane Brewing Company,15,15,96.0,India Pale Ale (IPA),96.0,0.822648,1.000000,1.000000
